In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import sys

In [2]:
import matplotlib
matplotlib.rcParams['figure.figsize'] = (10.0, 10.0)
matplotlib.rcParams['image.cmap'] = 'gray'

## Callback functions

In [11]:
'''
Gaussian Blue version
'''
# def EventMouseClick(event, x, y, flags, param): # occurs when mouse is clicked
#   '''
#   faster mouse event
#   '''
#   global radius_g, img
#   # convert radius_param to radius_g
#   radius_g = 15 + 5*radius_param
#   img_c = img.copy()
#   if event == cv2.EVENT_LBUTTONDOWN:
#     row, col, dim = img.shape
#     _src = img[max(0, y-radius_g//2+1): min(row, y+radius_g//2+1), max(0, x-radius_g//2+1): min(col, x+radius_g//2+1)].copy()
#     smooth_ksize = 75 # higher value, more smoothing, must be odd valued
#     _src = cv2.GaussianBlur(_src,(smooth_ksize,smooth_ksize), sigmaX = 0, sigmaY = 0)
    
#     # add gaussian noise
# #     gaussian = np.random.normal(0, 5, (_src.shape[0],_src.shape[1]))
# #     _src[:,:,0], _src[:,:,1], _src[:,:,2] = _src[:,:,0] + gaussian, _src[:,:,1] + gaussian, _src[:,:,2] + gaussian

#     _mask = np.zeros(_src.shape, dtype = np.uint8)
#     row, col, dim = _mask.shape
#     cv2.circle(_mask, (row//2,col//2), radius_g, (255,255,255), -1)
#     img = cv2.seamlessClone(_src, img_c, _mask, (x,y), cv2.MIXED_CLONE)
#   return

'''
Neighborhood search version
'''
def EventMouseClick(event, x, y, flags, param): # occurs when mouse is clicked
  '''
  Optimized to neighborhood search, has some bugs if clicked near border
  '''
  global radius_g, img, radius_param

  # convert radius_param to radius_g
  radius_g = 15 + 5*radius_param

  # make a copy of the image 
  img_c = img.copy()

  if event == cv2.EVENT_LBUTTONDOWN:
    row, col, dim = img.shape

    # Copy the square patch
    neighbor = img[max(0, y-radius_g*2): min(row, y+radius_g*2), max(0, x-radius_g*2): min(col, x+radius_g*2)].copy()

    # we use the score from this grayscale image to get the best neighborhood patch
    gn = cv2.cvtColor(neighbor, cv2.COLOR_BGR2GRAY)

    # Perform a search of the best patch which is inverse of the autofocus assignment
    # In the assignment, the best score = maximum score, here the best patch = minimum score
    # feed it radius_g x radius_g segments to the SML and get the one with the lowest score 
    nrow, ncol = gn.shape

    best_score = sum_modified_laplacian(gn[0:radius_g,0:radius_g])
    best_neighbor_rc = [[0,0]]

    # get the best neighborhood, store the coordinates
    for r in range(5, nrow-radius_g):
      for c in range(5, ncol-radius_g):
        score = sum_modified_laplacian(gn[r:r+radius_g,c:c+radius_g])
        if best_score > score:
          best_score = score
          best_neighbor_rc.pop()
          best_neighbor_rc.append([r,c])
          
    # grab the slice of the best neighbor
    best_r, best_c = best_neighbor_rc[0]
    best_neighbor = neighbor[best_r:best_r+radius_g, best_c:best_c+radius_g]
    
    _mask = np.zeros((radius_g, radius_g), dtype = np.uint8)
    mrow, mcol = _mask.shape
    cv2.circle(_mask, (mrow//2,mcol//2), radius_g//2, (255,255,255), -1)
    img = cv2.seamlessClone(best_neighbor, img_c, _mask, (x,y), cv2.NORMAL_CLONE)
  return

def sum_modified_laplacian(im):
    ###
    # Modified laplacian = laplacian with kernel [0,-1,0]
    #                                            [-1,4,-1]
    #                                            [0,-1,0]
    # focus_measurement = sum(abs(ML))
    if im is None:
        print('The image does not exist')
        return 0
    k = np.array([[0,-1,0],[-1,4,-1],[0,-1,0]], np.float32)
    ml = cv2.filter2D(im, cv2.CV_64F, k)
    ml = np.abs(ml)
    out = np.sum(ml)
    return out

def EventChangeRadius(*args):
  global radius_param
  radius_param = args[0]
  return

In [12]:
'''
For blemish removal, we have an open window.
1. Select a patch of skin, from the center, grab all images within the radius of the selected center of the blemish
2. Measure the smoothness of the patch and select the area with high smoothness.
'''
path = 'blemish.png'
img = cv2.imread(path, -1)

if img is None:
  print('Image is missing from the path')
# else:
#   plt.imshow(img[:,:,::-1])

win_name = 'Blemish'
scaletext = 'Radius'
radius_g = 0 # 5*(1 + radius_param) => conversion of trackbar value
radius_param = 2 # used by trackbar
maxRadius = 4

k = 0 # key reader for closing
source = img.copy() # keep a copy of the original image
cv2.namedWindow(winname = win_name)
cv2.setMouseCallback(win_name, EventMouseClick)
while k != 27: # ascii code for escape
  cv2.createTrackbar(scaletext, win_name, radius_param, maxRadius, EventChangeRadius)
  cv2.imshow(win_name, img)
  k = cv2.waitKey(25) & 0xff # read keyboard press
  if k == 99: # pressing 'c' refreshes the image source
    img = np.copy(source)
  
cv2.destroyAllWindows()